In [0]:
%pip install --upgrade databricks-sdk>=0.61.0 psycopg
%restart_python

In [0]:
%run ./0-Parameters

In [0]:
from databricks.sdk import WorkspaceClient
from databricks.sdk.errors.platform import NotFound
from databricks.sdk.service.apps import App, AppResource, AppResourceDatabase, AppResourceDatabaseDatabasePermission, AppResourceSqlWarehouse, AppResourceSqlWarehouseSqlWarehousePermission
from databricks.sdk.service.catalog import PermissionsChange, Privilege
import psycopg
import uuid

w = WorkspaceClient()

In [0]:
app_conf = App(
    name=APP_NAME,
    default_source_code_path=APP_SRC_CODE_PATH,
    description="SPARQL query frontend",
    # effective_user_api_scopes=["iam.current-user:read", "iam.access-control:read"],
    resources=[
        AppResource(
            name="database",
            database=AppResourceDatabase(
                instance_name=INSTANCE_NAME,
                database_name=PG_DATABASE_NAME,
                permission=AppResourceDatabaseDatabasePermission.CAN_CONNECT_AND_CREATE,
            ),
            description="Low latency serving of latest triples.",
        ),
        AppResource(
            name="sql_warehouse",
            sql_warehouse=AppResourceSqlWarehouse(
                id=WAREHOUSE_ID,
                permission=AppResourceSqlWarehouseSqlWarehousePermission.CAN_USE,
            ),
        ),
    ],
)

try:
    app = w.apps.get(APP_NAME)
    print(f"App {app.name} already exists")
except NotFound:
    app = w.apps.create_and_wait(app_conf)
    print(f"Created app {app.name}")

In [0]:
r1 = w.grants.update(
    "TABLE",
    SYNCED_TABLE_FULL_NAME,
    changes=[
        PermissionsChange(
            principal=app.service_principal_client_id,
            add=[Privilege.SELECT],
        )
    ],
)

r2 = w.grants.update(
    "TABLE",
    SRC_TABLE_FULL_NAME,
    changes=[
        PermissionsChange(
            principal=app.service_principal_client_id,
            add=[Privilege.SELECT],
        )
    ],
)

In [0]:
instance = w.database.get_database_instance(INSTANCE_NAME)
cred = w.database.generate_database_credential(
    request_id=str(uuid.uuid4()), instance_names=[instance.name]
)
current_user = (
    dbutils.notebook.entry_point.getDbutils()
    .notebook()
    .getContext()
    .userName()
    .getOrElse(None)
)

conn_conf = {
    "host": instance.read_write_dns,
    "port": 5432,
    "dbname": PG_DATABASE_NAME,
    "user": current_user,
    "password": cred.token,
    "sslmode": "require",
    "autocommit": True,
}

grants = f"""
GRANT USAGE ON SCHEMA {SYNCED_TABLE_SCHEMA} TO "{app.service_principal_client_id}";
GRANT SELECT ON TABLE {SYNCED_TABLE_SCHEMA}.{SYNCED_TABLE_NAME} TO "{app.service_principal_client_id}";
"""

with psycopg.connect(**conn_conf) as conn:
    with conn.cursor() as cur:
        cur.execute(grants)